In [2]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

In [3]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

In [4]:

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

In [5]:
def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  

In [6]:
class_names = ["toi", "song", "truoc", "nhan_vien", "gia_dinh", "test_toi", "test_song", "test_truoc", 
               "test_nhan_vien", "test_gia_dinh"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("data", cname))

# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors)
print("centers", kmeans.cluster_centers_.shape)

Load toi dataset
Load song dataset
Load truoc dataset
Load nhan_vien dataset
Load gia_dinh dataset
Load test_toi dataset
Load test_song dataset
Load test_truoc dataset
Load test_nhan_vien dataset
Load test_gia_dinh dataset
vectors (18599, 36)
centers (10, 36)
centers (10, 36)


In [7]:
models = {}
for cname in class_names:
    class_vectors = dataset[cname]
    # convert all vectors to the cluster index
    # dataset['one'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
    dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])
    hmm = hmmlearn.hmm.MultinomialHMM(
        n_components=15, random_state=0, n_iter=1000, verbose=True,
#         startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0]),
#         transmat_prior=np.array([
#             [0.1,0.5,0.1,0.1,0.1,0.1,],
#             [0.1,0.1,0.5,0.1,0.1,0.1,],
#             [0.1,0.1,0.1,0.5,0.1,0.1,],
#             [0.1,0.1,0.1,0.1,0.5,0.1,],
#             [0.1,0.1,0.1,0.1,0.1,0.5,],
#             [0.1,0.1,0.1,0.1,0.1,0.5,],
#         ]),
    )
    if cname[:4] != 'test':
        X = np.concatenate(dataset[cname])
        lengths = list([len(x) for x in dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
print("Training done")

training class toi
(2499, 1) [22, 36, 28, 39, 33, 23, 23, 21, 20, 23, 40, 48, 12, 36, 18, 25, 12, 22, 14, 23, 34, 22, 30, 10, 84, 47, 22, 29, 21, 25, 30, 17, 19, 31, 24, 27, 19, 38, 25, 21, 84, 24, 17, 17, 17, 16, 36, 17, 20, 18, 18, 32, 33, 29, 18, 47, 19, 20, 18, 24, 33, 16, 23, 31, 37, 17, 49, 31, 25, 30, 24, 82, 26, 17, 16, 57, 15, 24, 31, 16, 20, 20, 20, 49, 31, 34, 23, 22, 24, 39] 90


         1       -5702.7803             +nan
         2       -5004.4203        +698.3600
         3       -4960.6821         +43.7382
         4       -4880.2123         +80.4698
         5       -4710.5801        +169.6322
         6       -4395.0513        +315.5288
         7       -3998.5065        +396.5448
         8       -3702.6195        +295.8870
         9       -3539.2525        +163.3670
        10       -3393.3270        +145.9255
        11       -3194.8430        +198.4840
        12       -2960.8008        +234.0422
        13       -2826.1059        +134.6949
        14       -2743.4936         +82.6123
        15       -2701.0983         +42.3953
        16       -2674.1709         +26.9274
        17       -2650.1967         +23.9742
        18       -2629.5009         +20.6958
        19       -2612.8133         +16.6875
        20       -2598.8307         +13.9827
        21       -2584.5216         +14.3091
        22       -2567.1960         +17.3255
        23

training class song
(3110, 1) [25, 36, 32, 47, 31, 24, 22, 35, 54, 27, 35, 51, 34, 24, 25, 28, 17, 46, 32, 26, 31, 35, 30, 32, 28, 25, 31, 45, 30, 26, 43, 32, 45, 33, 31, 35, 41, 43, 33, 46, 32, 30, 47, 32, 47, 47, 42, 47, 41, 42, 33, 47, 36, 31, 24, 22, 36, 30, 33, 34, 32, 27, 44, 36, 45, 22, 30, 35, 31, 43, 35, 31, 54, 30, 45, 26, 25, 44, 22, 24, 17, 42, 26, 33, 42, 32, 31, 33, 43, 51] 90


         1       -7079.0588             +nan
         2       -6114.8663        +964.1925
         3       -6042.7833         +72.0830
         4       -5810.6554        +232.1279
         5       -5057.5018        +753.1536
         6       -4162.9663        +894.5355
         7       -3780.1828        +382.7836
         8       -3522.2460        +257.9367
         9       -3278.2274        +244.0186
        10       -3116.2224        +162.0050
        11       -3015.8795        +100.3429
        12       -2908.4387        +107.4407
        13       -2794.0390        +114.3997
        14       -2716.4385         +77.6004
        15       -2672.6261         +43.8124
        16       -2643.4747         +29.1515
        17       -2622.2678         +21.2069
        18       -2607.8899         +14.3779
        19       -2598.3269          +9.5630
        20       -2591.1131          +7.2138
        21       -2585.7155          +5.3976
        22       -2581.1851          +4.5304
        23

training class truoc
(2506, 1) [27, 20, 20, 41, 29, 31, 24, 26, 41, 21, 31, 26, 18, 23, 32, 23, 26, 21, 21, 37, 36, 26, 31, 15, 17, 19, 23, 26, 41, 25, 31, 31, 26, 31, 31, 26, 19, 35, 21, 22, 24, 20, 11, 31, 31, 22, 29, 33, 22, 29, 31, 22, 23, 24, 23, 31, 34, 21, 31, 26, 25, 20, 27, 19, 28, 27, 36, 23, 25, 35, 23, 31, 21, 41, 41, 36, 27, 61, 43, 43, 36, 35, 36, 20, 19, 19, 46, 31, 18, 26] 90


...

       367       -3780.1782          +0.1425
       368       -3780.0614          +0.1168
       369       -3779.9565          +0.1049
       370       -3779.8550          +0.1015
       371       -3779.7514          +0.1036
       372       -3779.6417          +0.1097
       373       -3779.5229          +0.1189
       374       -3779.3926          +0.1302
       375       -3779.2499          +0.1427
       376       -3779.0948          +0.1551
       377       -3778.9289          +0.1659
       378       -3778.7552          +0.1737
       379       -3778.5779          +0.1773
       380       -3778.4015          +0.1764
       381       -3778.2300          +0.1715
       382       -3778.0665          +0.1635
       383       -3777.9125          +0.1540
       384       -3777.7686          +0.1439
       385       -3777.6346          +0.1341
       386       -3777.5096          +0.1250
       387       -3777.3927          +0.1168
       388       -3777.2830          +0.1097
       389

training class gia_dinh
(4305, 1) [64, 51, 46, 55, 39, 43, 35, 66, 41, 41, 76, 47, 40, 46, 71, 45, 51, 40, 51, 41, 41, 53, 59, 47, 61, 37, 51, 51, 51, 47, 36, 44, 43, 46, 55, 46, 43, 39, 49, 44, 46, 51, 53, 37, 49, 33, 51, 51, 37, 46, 33, 41, 46, 66, 39, 37, 49, 45, 47, 31, 41, 41, 46, 45, 71, 43, 66, 55, 51, 56, 49, 51, 71, 33, 55, 35, 49, 41, 41, 51, 46, 51, 61, 45, 43, 61, 41, 41, 66, 39] 90


         1      -10130.3005             +nan
         2       -9080.0295       +1050.2710
         3       -8962.8823        +117.1472
         4       -8678.6489        +284.2335
         5       -8054.9673        +623.6816
         6       -7274.5550        +780.4123
         7       -6598.4565        +676.0985
         8       -5858.1208        +740.3357
         9       -5141.2850        +716.8359
        10       -4795.9583        +345.3266
        11       -4601.9510        +194.0073
        12       -4482.5870        +119.3640
        13       -4411.9558         +70.6312
        14       -4360.1413         +51.8145
        15       -4308.2286         +51.9127
        16       -4249.6719         +58.5568
        17       -4193.2681         +56.4038
        18       -4147.1051         +46.1630
        19       -4099.9464         +47.1588
        20       -4043.3716         +56.5748
        21       -3992.8313         +50.5403
        22       -3964.8820         +27.9493
        23

Training done


       130       -3905.6800          +0.0096


In [8]:
print("Testing")
miss = {}
test_name = { "test_toi", "test_song", "test_truoc", "test_nhan_vien", "test_gia_dinh"}
for true_cname in test_name:
    kt = 0
    for O in dataset[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }
        inverse = [(value, key) for key, value in score.items()]
        pre = max(inverse)[1]
        print(true_cname, score, pre)
        if pre != true_cname[5:]:
            kt +=1
    miss[true_cname] = kt
print(miss)

Testing
test_gia_dinh {'toi': -60.29343750988049, 'song': -261.0755129149988, 'truoc': -86.20954472029409, 'nhan_vien': -59.61287740272758, 'gia_dinh': -49.84458502446358} gia_dinh
test_gia_dinh {'toi': -56.95840776070965, 'song': -164.22113703613655, 'truoc': -72.02111397440986, 'nhan_vien': -70.46814916366459, 'gia_dinh': -47.605533575627106} gia_dinh
test_gia_dinh {'toi': -70.703876717082, 'song': -120.23862048244547, 'truoc': -144.26869233177362, 'nhan_vien': -86.39906723917133, 'gia_dinh': -41.46112615971998} gia_dinh
test_gia_dinh {'toi': -47.405436690244635, 'song': -104.63648070752413, 'truoc': -62.07717705390895, 'nhan_vien': -56.721174875149885, 'gia_dinh': -31.233688869034925} gia_dinh
test_gia_dinh {'toi': -79.35963147201795, 'song': -233.25398465838248, 'truoc': -174.76944983018652, 'nhan_vien': -105.39561868412991, 'gia_dinh': -46.73657873510931} gia_dinh
test_gia_dinh {'toi': -74.32891612943732, 'song': -273.96612315000436, 'truoc': -71.97219221418217, 'nhan_vien': -68.5

In [9]:
print("Testing")
miss = {}
for true_cname in class_names:
    kt = 0
    for O in dataset[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }
        inverse = [(value, key) for key, value in score.items()]
        pre = max(inverse)[1]
        print(true_cname, score, pre)
        if pre != true_cname:
            kt +=1
    miss[true_cname] = kt
print(miss)

Testing
toi {'toi': -25.256287408821606, 'song': -97.5070221415795, 'truoc': -48.86706689793773, 'nhan_vien': -37.18976927528191, 'gia_dinh': -48.5976831740657} toi
toi {'toi': -25.055416572147976, 'song': -68.44781952105785, 'truoc': -29.401514470521157, 'nhan_vien': -22.050134729919762, 'gia_dinh': -25.050733981489042} nhan_vien
toi {'toi': -29.334768127605486, 'song': -88.29474520435267, 'truoc': -30.158987452223222, 'nhan_vien': -46.59129413371964, 'gia_dinh': -34.066595490324175} toi
toi {'toi': -76.36697255160954, 'song': -260.21101095712504, 'truoc': -126.78686037774567, 'nhan_vien': -114.86641503480837, 'gia_dinh': -118.96723959387315} toi
toi {'toi': -15.614237623369007, 'song': -113.23578854752378, 'truoc': -17.95497146330187, 'nhan_vien': -17.04542284775574, 'gia_dinh': -27.32984113962093} toi
toi {'toi': -25.197068692652504, 'song': -41.67373428980185, 'truoc': -125.86870180458013, 'nhan_vien': -46.48165127859287, 'gia_dinh': -43.086308393444305} toi
toi {'toi': -17.0459485

toi {'toi': -22.776715476814612, 'song': -76.81491070535783, 'truoc': -40.15978438441838, 'nhan_vien': -20.46386016201131, 'gia_dinh': -23.24716882199822} nhan_vien
toi {'toi': -13.641380426206101, 'song': -203.7408451406628, 'truoc': -16.634115922078944, 'nhan_vien': -16.748627611972484, 'gia_dinh': -21.273309664393263} toi
toi {'toi': -40.478070417373644, 'song': -148.90125515764186, 'truoc': -319.32270234910527, 'nhan_vien': -81.11122837942787, 'gia_dinh': -104.75559771846622} toi
toi {'toi': -36.55591495440863, 'song': -150.96150970607613, 'truoc': -44.09617000712438, 'nhan_vien': -44.86388011186212, 'gia_dinh': -40.39219156798992} toi
toi {'toi': -13.258490068794156, 'song': -191.0675153693123, 'truoc': -56.99066700202114, 'nhan_vien': -21.68106488242848, 'gia_dinh': -17.68975215195827} toi
toi {'toi': -27.36106967252411, 'song': -215.58181818047365, 'truoc': -42.193832204896815, 'nhan_vien': -45.22449425023066, 'gia_dinh': -38.17784768657574} toi
toi {'toi': -21.213847531759175, 

song {'toi': -84.96891092842549, 'song': -32.57763060274791, 'truoc': -43.392636681288046, 'nhan_vien': -81.52045421337154, 'gia_dinh': -37.48506386613595} song
song {'toi': -52.72976854399083, 'song': -31.100536079294518, 'truoc': -33.65555524224572, 'nhan_vien': -57.48414984311797, 'gia_dinh': -53.113229325754546} song
song {'toi': -57.53662539379053, 'song': -43.73549032036023, 'truoc': -72.11525806172527, 'nhan_vien': -69.8280021046543, 'gia_dinh': -47.427678870597994} song
song {'toi': -63.444562941973835, 'song': -31.996353459646695, 'truoc': -40.092860495126395, 'nhan_vien': -66.41694576449358, 'gia_dinh': -40.980765478924276} song
song {'toi': -57.53662539379053, 'song': -43.73549032036023, 'truoc': -72.11525806172527, 'nhan_vien': -69.8280021046543, 'gia_dinh': -47.427678870597994} song
song {'toi': -60.01460106217038, 'song': -46.06302305999488, 'truoc': -49.732336559556806, 'nhan_vien': -69.49434676782947, 'gia_dinh': -47.07495662189157} song
song {'toi': -49.999342526773965

truoc {'toi': -21.89444931246274, 'song': -41.19836893500185, 'truoc': -13.723438127261883, 'nhan_vien': -18.156668196851747, 'gia_dinh': -38.41291082707909} truoc
truoc {'toi': -33.07574948133491, 'song': -28.186408611162353, 'truoc': -22.341872135766213, 'nhan_vien': -30.040774747302535, 'gia_dinh': -39.32662177374303} truoc
truoc {'toi': -53.817863860647556, 'song': -30.355654103715302, 'truoc': -28.394516055927078, 'nhan_vien': -59.265749125145675, 'gia_dinh': -37.71784917742414} truoc
truoc {'toi': -46.776879379071495, 'song': -38.26121386094467, 'truoc': -42.248658768057545, 'nhan_vien': -47.63068984424138, 'gia_dinh': -31.65657891561434} gia_dinh
truoc {'toi': -38.525354497444134, 'song': -115.43334669742367, 'truoc': -31.741978188464426, 'nhan_vien': -33.65630512687666, 'gia_dinh': -30.66880221256907} gia_dinh
truoc {'toi': -44.17608364223804, 'song': -212.4223616858048, 'truoc': -28.583569494625376, 'nhan_vien': -38.23989639713391, 'gia_dinh': -38.300446090946885} truoc
truoc 

truoc {'toi': -58.99828233365635, 'song': -55.69500940693637, 'truoc': -41.29287054575326, 'nhan_vien': -49.9896445242505, 'gia_dinh': -53.34609747712022} truoc
truoc {'toi': -47.25523600212296, 'song': -51.82276412287564, 'truoc': -22.942366436360953, 'nhan_vien': -31.496123118306688, 'gia_dinh': -53.59510105254783} truoc
truoc {'toi': -31.069472647093278, 'song': -136.01962345930684, 'truoc': -21.578768674728593, 'nhan_vien': -31.024480673633505, 'gia_dinh': -38.28120537695495} truoc
truoc {'toi': -25.468040314942925, 'song': -29.955899495380216, 'truoc': -21.210327804746377, 'nhan_vien': -28.122637983912995, 'gia_dinh': -24.376879675351862} truoc
truoc {'toi': -26.904496407103817, 'song': -147.93784294697323, 'truoc': -22.201081754468024, 'nhan_vien': -29.68652009413899, 'gia_dinh': -27.998448523085184} truoc
truoc {'toi': -28.23456072238079, 'song': -28.324887164508873, 'truoc': -19.740908904706675, 'nhan_vien': -42.12034210018785, 'gia_dinh': -29.039106198654277} truoc
truoc {'toi

nhan_vien {'toi': -37.09795734420193, 'song': -214.4633268181476, 'truoc': -51.81032403046122, 'nhan_vien': -25.572823353540397, 'gia_dinh': -35.87673780795252} nhan_vien
nhan_vien {'toi': -41.32761517939124, 'song': -71.55364908463906, 'truoc': -59.267038474280916, 'nhan_vien': -28.373633480625873, 'gia_dinh': -42.51754472993582} nhan_vien
nhan_vien {'toi': -143.74380815351395, 'song': -393.3494356440076, 'truoc': -80.58657500311061, 'nhan_vien': -48.946173759826195, 'gia_dinh': -54.04444659318915} nhan_vien
nhan_vien {'toi': -72.97473189831581, 'song': -253.64984394859997, 'truoc': -87.9219474976267, 'nhan_vien': -44.761398948645535, 'gia_dinh': -54.57512326612138} nhan_vien
nhan_vien {'toi': -53.64289402794385, 'song': -99.09472710331933, 'truoc': -50.12137729878638, 'nhan_vien': -25.823121866279227, 'gia_dinh': -40.389332865064155} nhan_vien
nhan_vien {'toi': -58.417321134093505, 'song': -72.97240019870227, 'truoc': -62.89796948012589, 'nhan_vien': -32.94261212712905, 'gia_dinh': -

nhan_vien {'toi': -37.211216279860366, 'song': -69.47065261632841, 'truoc': -55.76067835517282, 'nhan_vien': -30.946892871122426, 'gia_dinh': -39.07588912841927} nhan_vien
nhan_vien {'toi': -50.470691732399715, 'song': -80.80201647664231, 'truoc': -56.1499492406058, 'nhan_vien': -31.1532814989659, 'gia_dinh': -44.32274399163291} nhan_vien
nhan_vien {'toi': -41.073451416175956, 'song': -229.37863826723273, 'truoc': -83.03682687487958, 'nhan_vien': -32.86612631507357, 'gia_dinh': -46.70039446219901} nhan_vien
nhan_vien {'toi': -31.990551791265307, 'song': -84.7615611278825, 'truoc': -37.395607415831854, 'nhan_vien': -23.717452082790643, 'gia_dinh': -38.849067886701754} nhan_vien
nhan_vien {'toi': -58.45671914595667, 'song': -242.30248227555563, 'truoc': -90.61494912308927, 'nhan_vien': -45.91772381572703, 'gia_dinh': -75.35889662887176} nhan_vien
nhan_vien {'toi': -26.402832196125672, 'song': -144.8225147276719, 'truoc': -87.8555646386689, 'nhan_vien': -31.710732472189083, 'gia_dinh': -2

gia_dinh {'toi': -34.474228948962384, 'song': -183.1304811859642, 'truoc': -53.90132337340149, 'nhan_vien': -32.181695288043784, 'gia_dinh': -33.65590137039291} nhan_vien
gia_dinh {'toi': -60.90115167847213, 'song': -173.27628003955644, 'truoc': -79.34294908038076, 'nhan_vien': -67.52361424924536, 'gia_dinh': -41.57425148606913} gia_dinh
gia_dinh {'toi': -36.70713653743652, 'song': -82.58846655217305, 'truoc': -40.06334678050744, 'nhan_vien': -24.6355333113677, 'gia_dinh': -28.72642770129865} nhan_vien
gia_dinh {'toi': -57.51168726461343, 'song': -253.18240637222482, 'truoc': -68.05311211810296, 'nhan_vien': -51.357870819857446, 'gia_dinh': -36.995600085707025} gia_dinh
gia_dinh {'toi': -56.9555141749389, 'song': -204.91422201421318, 'truoc': -61.71752701586478, 'nhan_vien': -50.2813607464403, 'gia_dinh': -39.29066673481551} gia_dinh
gia_dinh {'toi': -65.07077039375568, 'song': -269.5318907814396, 'truoc': -120.8027866506851, 'nhan_vien': -80.53712738835975, 'gia_dinh': -54.33055415183

test_nhan_vien {'toi': -33.06013765985117, 'song': -107.29991761141119, 'truoc': -37.38582565966705, 'nhan_vien': -27.73108285111073, 'gia_dinh': -35.91332408389078} nhan_vien
test_nhan_vien {'toi': -60.83664952503594, 'song': -98.88550282247081, 'truoc': -68.79280911480593, 'nhan_vien': -47.69489470882618, 'gia_dinh': -74.55816453393672} nhan_vien
test_nhan_vien {'toi': -50.470691732399715, 'song': -80.80201647664231, 'truoc': -56.1499492406058, 'nhan_vien': -31.1532814989659, 'gia_dinh': -44.32274399163291} nhan_vien
test_gia_dinh {'toi': -60.29343750988049, 'song': -261.0755129149988, 'truoc': -86.20954472029409, 'nhan_vien': -59.61287740272758, 'gia_dinh': -49.84458502446358} gia_dinh
test_gia_dinh {'toi': -56.95840776070965, 'song': -164.22113703613655, 'truoc': -72.02111397440986, 'nhan_vien': -70.46814916366459, 'gia_dinh': -47.605533575627106} gia_dinh
test_gia_dinh {'toi': -70.703876717082, 'song': -120.23862048244547, 'truoc': -144.26869233177362, 'nhan_vien': -86.39906723917

In [10]:
hmm = hmmlearn.hmm.GMMHMM(n.components = 5, n.mix = 2, params = 'mct', init_params = 'mc')
# mcst trong do m la mean, c la xich ma, s la pi, t la A


SyntaxError: keyword can't be an expression (<ipython-input-10-0a4e9f96f589>, line 1)